In [1]:
package.path = '../?.lua;'..package.path
require 'torch'
require 'nn'
require 'image'

require 'densecap.DenseCapModel'
utils = require 'densecap.utils'
box_utils = require 'densecap.box_utils'
vis_utils = require 'densecap.vis_utils'

In [ ]:
-- load model
dtype, use_cudnn = utils.setup_gpus(0, 1)
print(dtype, use_cudnn)
checkpoint = '../data/models/densecap/densecap-pretrained-vgg16.t7'
model = torch.load(checkpoint).model
model:convert(dtype, use_cudnn)
model:evaluate()
-- model test option
model:setTestArgs{rpn_nms_thresh = 0.7, 
                  final_nms_thresh = 0.3,
                  num_proposals = 1000}

In [2]:
function view_box(box)
    x1 = torch.floor(box[1]-box[3]/2+1)
    y1 = torch.floor(box[2]-box[4]/2+1)
    x2 = x1 + box[3] - 1
    y2 = y1 + box[4] - 1
    itorch.image(img[{ {}, {y1, y2}, {x1, x2} }])
end
-- options
image_size = 720
img_path = '../imgs/elephant.jpg'
box_width = 2
-- read image and box
img = image.load(img_path, 3)

In [3]:
img = image.scale(img, image_size):float()
H, W = img:size(2), img:size(3)
-- convert img
img_caffe = img:view(1, 3, H, W)
img_caffe = img_caffe:index(2, torch.LongTensor{3,2,1}):mul(255)
vgg_mean = torch.FloatTensor{103.939, 116.779, 123.68}
vgg_mean = vgg_mean:view(1, 3, 1, 1):expand(1, 3, H, W)
img_caffe:add(-1, vgg_mean)

In [ ]:
out = model:extractAllFeatures(img_caffe:type(dtype))
boxes_scores, seqs, roi_codes, hidden_codes, captions = unpack(out)
print(boxes_scores:size(),boxes_scores:type())
print(roi_codes:size(), roi_codes:type())
print(hidden_codes:size(), hidden_codes:type())
print(seqs:size(), seqs:type())
print(captions[1], captions[2], captions[3], captions[4], captions[5])

In [ ]:
-- forward_test
boxes, scores, captions = model:forward_test(img_caffe:type(dtype))
print(boxes:size(), boxes:type())
print(captions[1], captions[2], captions[3], captions[4], captions[5])